In [ ]:
!pip install spacy
!pip install nltk

In [ ]:
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [ ]:
!pip install openpyxl


[notice] A new release of pip is available: 23.1.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import datetime
import nltk
import string
import os
import re
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
import spacy
from openpyxl.workbook import Workbook
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
Driver_path = r"C:\\Users\\user\\Desktop\\chromedriver.exe"
service = Service(Driver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
url = 'https://www.gutenberg.org/'

In [ ]:
driver.get(url)

In [ ]:
bookshelve = driver.find_element(By.XPATH,'/html/body/div[1]/div/ul[1]/li[2]/a')
bookshelve.click()

In [ ]:
#XPATHS
educational_XPath = '/html/body/div[1]/div/div[1]/ul[3]/li[1]/a'
education_books_XPath = '//*[@id="content"]/div[2]/div/ul/li[7]'
crime_XPath = '/html/body/div[1]/div/div[1]/ul[2]/li[5]/a'
crime_fiction_XPath = '//*[@id="content"]/div[2]/div/ul/li[7]/a/span[2]/span[1]'

In [ ]:
educational_book_shelf = driver.find_element(By.XPATH,educational_XPath)
educational_book_shelf.click()

In [ ]:
education_books = driver.find_element(By.XPATH,education_books_XPath)
education_books.click()

In [ ]:
books = driver.find_elements(By.CLASS_NAME,'booklink')

In [ ]:
links=[]
check = 0
for i in books:
    links.append(i.find_element(By.CLASS_NAME,'link').get_attribute('href'))
    check=check+1
    if check==3:
        break

In [ ]:
books=[]
for link in links:
    driver.get(link)
    time.sleep(2)
    read = driver.find_element(By.XPATH,'//*[@id="download"]/div/table/tbody/tr[8]/td[2]/a')
    if read.text.strip()=='Plain Text UTF-8':
        read.click()
    else:
        break
    book = driver.find_element(By.XPATH,'/html/body/pre')
    books.append(book.text.strip())

In [ ]:
driver.get(url)

In [ ]:
bookshelve = driver.find_element(By.XPATH,'/html/body/div[1]/div/ul[1]/li[2]/a')
bookshelve.click()

In [ ]:
crime_book_shelf = driver.find_element(By.XPATH, crime_XPath)
crime_book_shelf.click()

In [ ]:
crime_fiction = driver.find_element(By.XPATH, crime_fiction_XPath)
crime_fiction.click()

In [ ]:
book = driver.find_elements(By.CLASS_NAME,'booklink')
check = 0
for i in book:
    links.append(i.find_element(By.CLASS_NAME,'link').get_attribute('href'))
    check=check+1
    if check==3:
        break

In [ ]:
j = 1
for url in links[3:]:
    driver.get(url)
    time.sleep(4)
    text_link = driver.find_element(by=By.XPATH,value='//*[@id="download"]/div/table/tbody/tr[7]/td[2]/a')
    if text_link.text.strip()=='Plain Text UTF-8':
        text_link.click()
    else:
        break
    book = driver.find_element(By.XPATH,'/html/body/pre')
    books.append(book.text.strip())
    print("Book Scrapped : " , j )
    j = j + 1

Book Scrapped :  1
Book Scrapped :  2
Book Scrapped :  3


In [ ]:
print(len(books))

5


In [ ]:
books

In [ ]:
#Raw Data to txt file
file_path = 'RawData.txt'
with open(file_path, 'w') as file:
    for item in books:
        file.write(f"{item}\n")

In [ ]:
def preprocess_text(text, alpha=1.0):
    # Tokenization: Tokenize the text into words or tokens using NLTK
    tokens = nltk.word_tokenize(text)
    # Lowercasing: Convert all tokens to lowercase
    tokens = [token.lower() for token in tokens]

    tokens = [token for token in tokens if token.isalnum() or (token in string.punctuation and "'" in token)]

    # Join the preprocessed tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

def remove_space(sentence):
    return re.sub(r"``(.*?)''", r"\1", sentence)

def Preprocess_ScrapedData(books):
    books =[i.replace('\n',' ') for i in books]
    books = [i.strip() for i in books]
    intro = "This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook."

    sentences_per_book = []
    for book in books:
    # Remove the intro text
        cleaned_text = book.replace(intro, '')

    # Process the cleaned text using Spacy
        nlp.max_length = 2000000
        doc = nlp(cleaned_text)

    # Split the text into sentences
        sentences = [sent.text.strip() for sent in doc.sents]
        sentences_per_book.append(sentences)

    lines = []
    for sentence in sentences_per_book:
        for line in sentence:
            if line == ' ':
                continue
            if line not in lines:
                lines.append(line.strip())

    processed_lines = [preprocess_text(sentence) for sentence in lines]

    processed_sentences = [remove_space(sentence) for sentence in processed_lines]

    processed_text = [line for line in processed_sentences if (len(line)>30) and ('http //www' not in line) and ('project gutenberg' not in line) and ('ebook' not in line)]

    processed_text = [line.replace("``",'').strip() for line in processed_text]
    processed_text = [re.sub(r'\d+', '', line).strip() for line in processed_text]

    filtered_sentences = []
    [filtered_sentences.append(i) for i in processed_text if i not in filtered_sentences]

    return filtered_sentences

In [ ]:
filtered_sentences = Preprocess_ScrapedData(books)
len(filtered_sentences)

22665

In [ ]:
df_filtered = pd.DataFrame({'Sentences': filtered_sentences})

In [ ]:
df_filtered

,Sentences
0,montessori own handbook produced by alicia wil...
1,illustration maria montessori dr
2,montessori own handbook by maria montessori au...
3,with new york frederick stokes company publishers
4,copyright frederick stokes company
...,...
22660,dieusy surveyed them for a moment with blank a...
22661,then with swift intelligence grasped the fact ...
22662,he sprang upon them tore them asunder fell hea...
22663,guerchard dashed to the door tried it and foun...


In [ ]:
#TFIDF
tfidf_vectorizer = TfidfVectorizer()


tfidf_matrix = tfidf_vectorizer.fit_transform(df_filtered['Sentences'])


df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
df_tfidf

,aback,abandon,abandoned,abandoning,abbey,abbreviated,aberration,abide,abiding,abilities,...,zeus,zigzags,zimmerman,zone,zossimov,æssthetic,æsthetic,æsthetically,æsthetics,êtes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Preprocessed Data to excel file
excel_file_path = 'PreprocssedData.xlsx'
df_filtered.to_excel(excel_file_path, index=False)

In [ ]:
#BIGRAMS
sentences_tokenized = [word_tokenize(sentence.lower()) for sentence in filtered_sentences]
sentences_bigrams = [list(bigrams(tokens)) for tokens in sentences_tokenized]

structured_bigrams = []

for i, sentence_bigrams in enumerate(sentences_bigrams, start=1):
    for bigram in sentence_bigrams:
        structured_bigrams.append({
            'Sentence': f"Sentence {i}",
            'Bigram': ' '.join(bigram)
        })

df = pd.DataFrame(structured_bigrams)

print(df)

              Sentence             Bigram
0           Sentence 1     montessori own
1           Sentence 1       own handbook
2           Sentence 1  handbook produced
3           Sentence 1        produced by
4           Sentence 1          by alicia
...                ...                ...
390483  Sentence 22665        was rolling
390484  Sentence 22665   rolling smoothly
390485  Sentence 22665        smoothly to
390486  Sentence 22665               to a
390487  Sentence 22665        a honeymoon

[390488 rows x 2 columns]


In [ ]:
#Bigram to excel file
excel_file_path = 'BigramsData.xlsx'
df.to_excel(excel_file_path, index=False)